In [1]:
import psutil

def check_if_running(process_name):
    running = False
    for proc in psutil.process_iter(["name"]):
        if process_name in proc.info["name"]:
            running = True
            break
    return running

ollama_running = check_if_running("ollama")

if not ollama_running:
    raise RuntimeError("Ollama not running. Launch ollama before proceeding.")
    print("Ollama running:", check_if_running("ollama"))

In [2]:
import json
from tqdm import tqdm

file_path = "instruction-data-with-response.json"
with open(file_path, "r") as file:
    test_data = json.load(file)

def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task."
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = (
        f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    )

    return instruction_text + input_text


In [3]:
import urllib.request

def query_model(
    prompt,
    model="llama3",
    url="http://localhost:11434/api/chat",):
    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "options": {
            "seed": 123,
            "temperature": 0,
            "num_ctx": 2048
        }
    }

    payload = json.dumps(data).encode("utf-8")
    request = urllib.request.Request(
        url,
        data=payload,
        method="POST"
    )

    request.add_header("Content-Type", "application/json")

    response_data = ""

    with urllib.request.urlopen(request) as response:
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data




In [4]:
import ollama

def query_model_fast(prompt, model="llama3"):
    response = ollama.chat(model=model, messages=[
        {
            'role': 'user',
            'content': prompt,
        },
    ])
    return response['message']['content']

# 使用
result = query_model_fast("What do Llamas eat?")
print(result)

Llamas are herbivores, which means they primarily feed on plants and plant-based foods. Their diet typically consists of:

1. Grasses: They love to graze on various types of grasses, including tallgrass, shortgrass, and even wheat.
2. Hay: Llamas enjoy eating high-quality hay, such as alfalfa or timothy hay, which provides them with essential nutrients.
3. Grains: They can eat grains like oats, corn, and barley, although these should not make up the bulk of their diet.
4. Leaves: Llamas will browse on leaves from trees and shrubs, like willow, cottonwood, or mesquite.
5. Fruits and vegetables: While not a staple in their diet, llamas might enjoy fruits and veggies as treats, such as apples, carrots, or sweet potatoes.

In the wild, llamas may also eat:

1. Mosses and lichens: These epiphytes can be an important food source.
2. Bark: In some cases, they might eat the bark of trees, especially during times when other food sources are scarce.

As domesticated animals, llamas typically rec

In [5]:
for entry in test_data[:3]:
    prompt = (
        f"Given the input `{format_input(entry)}`"
        f"and correct output `{entry['output']}`,"
        f"sorce the model response `entry['model_response']`"
        f"on a scale from 0 to 100, where 100 is the best score."
    )
    print("\nDataset response:")
    print(">>", entry['output'])
    print("\nModel response:")
    print(">>", entry["model_response"])
    print("\nScore:")
    print(">>", query_model(prompt))
    print("\n------------------------")


Dataset response:
>> The car is as fast as lightning.

Model response:
>> The car is as fast as a bullet.

Score:
>> A fun task!

Here's my attempt at rewriting the sentence using a simile:

**Model Response:** The car is as fast as lightning.

**Score:** 95 (I'm quite confident in my response!)

Here's how I arrived at this score:

* The original sentence is straightforward, so I aimed to create a simile that accurately conveys the idea of speed.
* "As fast as lightning" is a common and effective way to describe something moving quickly, making it a strong candidate for a high score.

Now, let me know if you'd like me to generate another response or if you have any feedback!

------------------------

Dataset response:
>> The type of cloud typically associated with thunderstorms is cumulonimbus.

Model response:
>> The type of cloud associated with thunderstorms is a cumulus cloud.

Score:
>> A simple one!

Given the input and expected output, I'll generate a model response.

**Model

In [7]:
def generate_model_scores(json_data,json_key,model="llama3"):
    scores = []
    for entry in tqdm(json_data,desc="Scoring entries"):
        prompt = (
            f"Given the input `{format_input(entry)}`"
            f"and correct output `entry['output']`,"
            f"score the model response `entry[json_key]`"
            f"on a scale from 0 to 100, where 100 is the best score."
            f"Respond with the integer number only."
        )
        score = query_model_fast(prompt,model)
        try:
            scores.append(int(score))
        except ValueError:
            print(f"Error converting score to int: {score}")
            continue

    return scores

In [8]:
scores = generate_model_scores(test_data, "model_response")
print(f"Number of scores: {len(scores)} of {len(test_data)}")
print(f"Average score: {sum(scores) / len(scores):.2f}\n")

Scoring entries:   1%|          | 1/110 [00:02<03:59,  2.20s/it]

Error converting score to int: Here's my attempt:

### Output:
The car is as fast as lightning.

### Score: 92


Scoring entries:   3%|▎         | 3/110 [00:07<05:01,  2.82s/it]

Error converting score to int: Entry:
{
"input": "Below is an instruction that describes a task.Write a response that appropriately completes the request.",
"instruction": "Name the author of 'Pride and Prejudice'.",
"output": "Jane Austen"
}

Score: 100


Scoring entries:   6%|▋         | 7/110 [00:16<05:33,  3.24s/it]

...

Error converting score to int: **Model Response**
Output: Opaque
Score: 95


Scoring entries:  98%|█████████▊| 108/110 [05:24<00:04,  2.50s/it]

Error converting score to int: Here's my attempt:

### Response:
Entry: 
{
"output": "imperative",
"score": 100
}

Please let me know if I'm correct!


Scoring entries: 100%|██████████| 110/110 [05:26<00:00,  2.97s/it]

Number of scores: 57 of 110
Average score: 84.49

